# <span style='color:#8b46b3'><center style='background:#ff7ea8;border-radius:0px 100px;padding:25px'> 🌸📉Parallel Programming with Python📈🌺</center></span>



## Code without any parallelization

In [ ]:
import math
import time

def calculate_pi(N):
    # Width of each small rectangle
    delta_x = 1.0 / N
    sum_area = 0.0

    # Calculate area using the sum of rectangles
    for i in range(N):
        x_i = i * delta_x
        f_x_i = math.sqrt(1 - x_i**2)  # f(x) = sqrt(1 - x^2)
        sum_area += f_x_i * delta_x

    # The total area under the curve is an approximation for pi/4, so multiply by 4
    pi_approx = sum_area * 4
    return pi_approx

# Number of rectangles
N = 1000000

# Start timing
start_time = time.time()

pi_value = calculate_pi(N)

# Stop timing
end_time = time.time()

# Calculate execution time
execution_time = end_time - start_time

print(f"Approximation of π with N={N} {pi_value}")
print(f"Execution time: {execution_time} seconds")


Approximation of π with N=100000000 3.1415926735892157
Execution time: 34.828962564468384 seconds


## Parallel computing via multiprocessing

In [ ]:
import math
from multiprocessing import Pool
import time  # Import the time module

def calculate_area(x):
    """ Function to calculate the area of a single rectangle """
    return math.sqrt(1 - x**2) * delta_x

def parallel_pi_calculation(N, num_processes):
    """ Function to calculate pi using multiple processes """
    global delta_x
    delta_x = 1.0 / N  # Width of each rectangle

    # Create a list of x values where the height of the rectangle will be calculated
    x_values = [i * delta_x for i in range(N)]

    # Start timing before the calculation starts
    start_time = time.time()

    # Create a pool of processes and calculate areas
    with Pool(processes=num_processes) as pool:
        areas = pool.map(calculate_area, x_values)

    # Stop timing after the calculation is complete
    end_time = time.time()

    # Calculate total execution time
    execution_time = end_time - start_time

    # Sum the areas calculated by different processes and multiply by 4 to get pi
    total_area = sum(areas)
    pi_approx = 4 * total_area

    return pi_approx, execution_time

# Number of rectangles and number of processes
N = 10000000
num_processes = 8  # Adjust this according to your system's CPU cores

# Calculate pi and get the execution time
pi_value, time_taken = parallel_pi_calculation(N, num_processes)
print(f"Approximated value of pi: {pi_value}")
print(f"Execution time: {time_taken} seconds")


Approximated value of pi: 3.1415928535523587
Execution time: 6.970126628875732 seconds


## Distributed parallel computing via mi4py

In [ ]:
pip install mpi4py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 28.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-3.1.6-cp310-cp310-linux_x86_64.whl size=2746325 sha256=105e13f44fedb78f479014efd664540c11c2ec9198d7d5bafd765f3701d90a89
  Stored in directory: /root/.cache/pip/wheels/4c/ca/89/8fc1fb1c620afca13bb41c630b1f948bbf446e0aaa4b762e10
Successfully built mpi4py


In [ ]:
from mpi4py import MPI
import numpy as np
import time

def f(x):
    return np.sqrt(1 - x**2)

def main():
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    size = comm.Get_size()

    # Total number of rectangles
    N = 1000000
    delta_x = 1.0 / N

    # Each process will handle a subset of the total range
    local_n = N // size
    local_a = rank * local_n * delta_x
    local_b = local_a + local_n * delta_x

    local_sum = 0.0

    # Start timing for each process
    start_time = time.time()

    for i in range(local_n):
        x = local_a + (i + 0.5) * delta_x
        local_sum += f(x) * delta_x

    # Gather all local sums into the root process
    total_sum = comm.reduce(local_sum, op=MPI.SUM, root=0)

    # End timing for each process
    end_time = time.time()

    # Calculate and print the final result and execution time in the root process
    if rank == 0:
        pi_approx = total_sum * 4
        execution_time = end_time - start_time
        print(f"Approximated value of pi: {pi_approx}")
        print(f"Execution time: {execution_time} seconds")

if __name__ == '__main__':
    main()


Approximated value of pi: 3.1415926539343633
Execution time: 1.480806589126587 seconds
